# **Preprocess dataset**
We will preprocess Edge-IIoT dataset in this section

In [1]:
import numpy as np
import pandas as pd
import os

import csv
import matplotlib.pyplot as plt

from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, AvgPool1D, Flatten, Dense, Dropout, Softmax
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras.utils import to_categorical, plot_model

from sklearn.utils import shuffle

In [ ]:
os.chdir("D:\\IDSGAN\\IDSGAN-main\\IDSGAN\\Dataset\\Edge-IIoTset\\")
df = pd.read_csv("D:\\IDSGAN\\IDSGAN-main\\IDSGAN\\Dataset\\Edge-IIoTset\\DNN-EdgeIIoT-dataset.csv", low_memory=False)


In [41]:
#Combine dataset for pretrain model
file1 = pd.read_csv('D:\\IDSGAN\\IDSGAN-main\\IDSGAN\\Dataset\\Edge-IIoTset\\DNN-EdgeIIoT-dataset.csv')
drop_columns = ["frame.time", "ip.src_host", "ip.dst_host", "arp.src.proto_ipv4","arp.dst.proto_ipv4",
                "http.file_data","http.request.full_uri","icmp.transmit_timestamp",
                "http.request.uri.query", "tcp.options","tcp.payload","tcp.srcport",
                "tcp.dstport", "udp.port", "mqtt.msg"]

file1.drop(drop_columns, axis=1, inplace=True)
file1.dropna(axis=0, how='any', inplace=True)
file1.drop_duplicates(subset=None, keep="first", inplace=True)

file2 = pd.read_csv('D:\\IDSGAN\\IDSGAN-main\\IDSGAN\\Dataset\\Edge-IIoTset\\synthetic_data_CS.csv')
df = pd.concat( [file1,file2],ignore_index=True)
df.to_csv('combined_file.csv', index=False)
df = pd.read_csv("combined_file.csv", low_memory=False)

C:\Users\Ngoc Thien\AppData\Local\Temp\ipykernel_7892\1289634901.py:2: DtypeWarning: Columns (2,3,6,11,13,14,15,16,17,31,32,34,39,45,51,54,55) have mixed types. Specify dtype option on import or set low_memory=False.
  file1 = pd.read_csv('D:\\IDSGAN\\IDSGAN-main\\IDSGAN\\Dataset\\Edge-IIoTset\\DNN-EdgeIIoT-dataset.csv')
C:\Users\Ngoc Thien\AppData\Local\Temp\ipykernel_7892\1289634901.py:12: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  file2 = pd.read_csv('D:\\IDSGAN\\IDSGAN-main\\IDSGAN\\Dataset\\Edge-IIoTset\\synthetic_data_CS.csv')


In [ ]:
print(df["Attack_type"].unique())

['Normal' 'MITM' 'Uploading' 'Ransomware' 'SQL_injection' 'DDoS_HTTP'
 'DDoS_TCP' 'Password' 'Port_Scanning' 'Vulnerability_scanner' 'Backdoor'
 'XSS' 'Fingerprinting' 'DDoS_UDP' 'DDoS_ICMP']


In [39]:
drop_columns = ["frame.time", "ip.src_host", "ip.dst_host", "arp.src.proto_ipv4","arp.dst.proto_ipv4",
                "http.file_data","http.request.full_uri","icmp.transmit_timestamp",
                "http.request.uri.query", "tcp.options","tcp.payload","tcp.srcport",
                "tcp.dstport", "udp.port", "mqtt.msg"]

df.drop(drop_columns, axis=1, inplace=True)
df.dropna(axis=0, how='any', inplace=True)
df.drop_duplicates(subset=None, keep="first", inplace=True)
df = shuffle(df) #shuffle row of df

In [42]:
#one-hot encoding
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = f"{name}-{x}"
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)

encode_text_dummy(df,'http.request.method')
encode_text_dummy(df,'http.referer')
encode_text_dummy(df,"http.request.version")
encode_text_dummy(df,"dns.qry.name.len")
encode_text_dummy(df,"mqtt.conack.flags")
encode_text_dummy(df,"mqtt.protoname")
encode_text_dummy(df,"mqtt.topic")


In [43]:
print(len(df.columns))

97


In [ ]:
# Apply Median Filtering to all features
window_size_median = 3  # Window size for median filtering
for feature in df.columns:
    if feature != 'Attack_type':  # Exclude the label column
        df[feature + '_median_filtered'] = df[feature].rolling(window=window_size_median, center=True).median()

# Apply Standard Deviation-based Filtering to all features
threshold_std = 3  # Threshold for standard deviation-based filtering
for feature in df.columns:
    if feature != 'Attack_type':  # Exclude the label column
        mean = df[feature].mean()
        std = df[feature].std()
        upper_bound = mean + threshold_std * std
        lower_bound = mean - threshold_std * std
        df[feature + '_std_filtered'] = df[feature].apply(lambda x: x if (lower_bound <= x <= upper_bound) else None)

# Drop original features that were filtered
df.drop(df.columns[df.columns.str.endswith('_filtered')], axis=1, inplace=True)

In [45]:
df.to_csv('preprocessed_combined.csv', index=False)

In [ ]:
#df = pd.read_csv('preprocessed.csv', low_memory=False)
# df = pd.read_csv('preprocessed_combined.csv', low_memory=False)
df = pd.read_csv('preprocessed_DNN.csv', low_memory=False)
label_col = "Attack_type"
# X = preprocessed_df.drop([label_col], axis=1)
# y = preprocessed_df[label_col]
X = df.drop([label_col], axis=1)
y = df[label_col]
print("X: ", X.shape)
print("y: ", y.shape)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

test_shape = X_test.shape[0]
print("Training samples: ", test_shape)

del X
del y
print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test: ", y_test.shape)

X:  (1909671, 96)
y:  (1909671,)
Training samples:  381935
X_train:  (1527736, 96)
y_train:  (1527736,)
X_test:  (381935, 96)
y_test:  (381935,)


In [3]:
# Converting categorical value in "Attack_type" to integer value
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train =  label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

label_encoder.classes_

#Normalize
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()
X_train =  min_max_scaler.fit_transform(X_train)
X_test = min_max_scaler.transform(X_test)

In [5]:
X_train.shape

(1854243, 96)

In [4]:
num_classes = len(np.unique(y_train))
num_classes

15

In [5]:
# One-hot y_train and y_test
from  tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)

# **CNN-based IDS**

**Have 3 layer**

In [10]:
# Convert X_train and X_test dimension to 3-dimensional (row, column, channel)
X_train_CNN_3_layers = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_CNN_3_layers = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [51]:
print(X_train_CNN_3_layers.shape)
print(X_test_CNN_3_layers.shape)
input_shape = X_train_CNN_3_layers.shape[1:]
print(input_shape)

(1854243, 96, 1)
(463561, 96, 1)
(96, 1)


In [52]:
#CNN-based IDS model with 3 layers
#A deep learning approach for intrusion detection in Internet of Things using focal loss function
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

model = Sequential()

# Convolutional layers
model.add(Conv1D(32, 3, activation='relu', input_shape=input_shape))
model.add(MaxPooling1D(2))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))
model.summary()

d:\IDSGAN\IDSGAN-main\IDSGAN\myenv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 94, 32)         │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 47, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1504)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 15)             │        22,575 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,703 (88.68 KB)

 Trainable params: 22,703 (88.68 KB)

 Non-trainable params: 0 (0.00 B)

In [53]:
import tensorflow as tf
opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss= tf.keras.metrics.categorical_crossentropy,
                  metrics=['accuracy'])


from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, mode="min", verbose=1, min_lr=0)
#plotlosses = PlotLossesKeras()
call_backs = [lr_reduce]
EPOCHS = 15
BATCH_SIZE = 256

history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    validation_split=0.1,
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE,
                    callbacks=call_backs,
                    #class_weight=class_weights,
                    verbose=1)
# model.save('CNN_3_layers.h5')
model.save('CNN_3_layers_pretrain.h5')

Epoch 1/15
7244/7244 ━━━━━━━━━━━━━━━━━━━━ 24s 3ms/step - accuracy: 0.8550 - loss: 0.4891 - val_accuracy: 0.8715 - val_loss: 0.3985 - learning_rate: 0.0010
Epoch 2/15
7244/7244 ━━━━━━━━━━━━━━━━━━━━ 26s 4ms/step - accuracy: 0.8721 - loss: 0.3958 - val_accuracy: 0.8724 - val_loss: 0.3934 - learning_rate: 0.0010
Epoch 3/15
7244/7244 ━━━━━━━━━━━━━━━━━━━━ 22s 3ms/step - accuracy: 0.8729 - loss: 0.3919 - val_accuracy: 0.8737 - val_loss: 0.3912 - learning_rate: 0.0010
Epoch 4/15
7244/7244 ━━━━━━━━━━━━━━━━━━━━ 46s 6ms/step - accuracy: 0.8737 - loss: 0.3876 - val_accuracy: 0.8766 - val_loss: 0.3871 - learning_rate: 0.0010
Epoch 5/15
7244/7244 ━━━━━━━━━━━━━━━━━━━━ 58s 8ms/step - accuracy: 0.8748 - loss: 0.3839 - val_accuracy: 0.8760 - val_loss: 0.3825 - learning_rate: 0.0010
Epoch 6/15
7244/7244 ━━━━━━━━━━━━━━━━━━━━ 57s 8ms/step - accuracy: 0.8762 - loss: 0.3805 - val_accuracy: 0.8754 - val_loss: 0.3815 - learning_rate: 0.0010
Epoch 7/15
7244/7244 ━━━━━━━━━━━━━━━━━━━━ 80s 8ms/step - accuracy: 0.8

**Have 5 layers**

In [9]:
# Convert X_train and X_test dimension to 3-dimensional (row, column, channel)
X_train_CNN_5_layers = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_CNN_5_layers = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [56]:
print(X_train_CNN_5_layers.shape)
print(X_test_CNN_5_layers.shape)

(1854243, 96, 1)
(463561, 96, 1)


In [57]:
#CNN-based IDS model with 5 layers
#https://www.researchgate.net/publication/341555600_CNID_Research_of_Network_Intrusion_Detection_Based_on_Convolutional_Neural_Network
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, LSTM, Dropout

model = Sequential()

# Convolutional layers
model.add(Conv1D(32, 3, activation='relu', input_shape=input_shape))
model.add(MaxPooling1D(2))
model.add(Conv1D(64, 3, activation='relu'))
model.add(MaxPooling1D(2))

model.add(Flatten())

model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

#Summary
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_1 (Conv1D)               │ (None, 94, 32)         │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 47, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 45, 64)         │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 22, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1408)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1408)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │        21,135 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 27,471 (107.31 KB)

 Trainable params: 27,471 (107.31 KB)

 Non-trainable params: 0 (0.00 B)

In [58]:
import tensorflow as tf
opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss= tf.keras.metrics.categorical_crossentropy,
                  metrics=['accuracy'])


from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, mode="min", verbose=1, min_lr=0)
#plotlosses = PlotLossesKeras()
call_backs = [lr_reduce]
EPOCHS = 15
BATCH_SIZE = 256

history = model.fit(X_train_CNN_5_layers, y_train,
                    validation_data=(X_test_CNN_5_layers, y_test),
                    validation_split=0.1,
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE,
                    callbacks=call_backs,
                    #class_weight=class_weights,
                    verbose=1)
# model.save('CNN_5_layers.h5')
model.save('CNN_5_layers_pretrain.h5')

Epoch 1/15
7244/7244 ━━━━━━━━━━━━━━━━━━━━ 48s 6ms/step - accuracy: 0.8565 - loss: 0.4708 - val_accuracy: 0.8717 - val_loss: 0.3888 - learning_rate: 0.0010
Epoch 2/15
7244/7244 ━━━━━━━━━━━━━━━━━━━━ 49s 7ms/step - accuracy: 0.8726 - loss: 0.3903 - val_accuracy: 0.8753 - val_loss: 0.3812 - learning_rate: 0.0010
Epoch 3/15
7244/7244 ━━━━━━━━━━━━━━━━━━━━ 49s 7ms/step - accuracy: 0.8744 - loss: 0.3841 - val_accuracy: 0.8767 - val_loss: 0.3769 - learning_rate: 0.0010
Epoch 4/15
7244/7244 ━━━━━━━━━━━━━━━━━━━━ 50s 7ms/step - accuracy: 0.8762 - loss: 0.3779 - val_accuracy: 0.8777 - val_loss: 0.3708 - learning_rate: 0.0010
Epoch 5/15
7244/7244 ━━━━━━━━━━━━━━━━━━━━ 50s 7ms/step - accuracy: 0.8782 - loss: 0.3729 - val_accuracy: 0.8802 - val_loss: 0.3656 - learning_rate: 0.0010
Epoch 6/15
7244/7244 ━━━━━━━━━━━━━━━━━━━━ 52s 7ms/step - accuracy: 0.8802 - loss: 0.3674 - val_accuracy: 0.8845 - val_loss: 0.3612 - learning_rate: 0.0010
Epoch 7/15
7244/7244 ━━━━━━━━━━━━━━━━━━━━ 51s 7ms/step - accuracy: 0.8

**Have 7 layers**

In [43]:
X_train.shape

(1527736, 96)

In [8]:
X_train_CNN_7_layers = np.zeros((X_train.shape[0], 32 * 32))
X_train_CNN_7_layers[:, :96] = X_train  # Fill in the first 96 elements

# Reshape to (390500, 32, 32, 1) for the CNN
X_train_CNN_7_layers = X_train_CNN_7_layers.reshape(-1, 32, 32, 1)

X_test_CNN_7_layers = np.zeros((X_test.shape[0], 32 * 32))
X_test_CNN_7_layers[:, :96] = X_test  # Fill in the first 97 elements

# Reshape to (390500, 32, 32, 1) for the CNN
X_test_CNN_7_layers = X_test_CNN_7_layers.reshape(-1, 32, 32, 1)

In [7]:
#CNN-based IDS model with 7 layers
#An Intrusion Detection System Based on Convolutional Neural Network
# from tensorflow.keras.layers import Reshape
# from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, LSTM

# model = Sequential()

# # Convolutional layers
# model.add(Conv1D(32, 3, activation='relu', input_shape=input_shape))
# model.add(MaxPooling1D(2))
# model.add(Conv1D(64, 3, activation='relu'))
# model.add(MaxPooling1D(2))
# # model.add(Conv1D(128, 3, activation='relu'))
# # model.add(MaxPooling1D(2))

# # Flatten output from Convolutional layers
# model.add(Flatten())


# # Dense layers for classification
# model.add(Dense(64, activation='relu'))
# model.add(Dense(num_classes, activation='softmax'))

# model.summary()
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense

# Define the model
model = Sequential()

# C1: Convolutional Layer
model.add(Conv2D(6, kernel_size=(5, 5), activation='relu', input_shape=(32, 32, 1)))

# S2: Subsampling (Pooling) Layer
model.add(AveragePooling2D(pool_size=(2, 2), strides=2))

# C3: Convolutional Layer
model.add(Conv2D(16, kernel_size=(5, 5), activation='relu'))

# S4: Subsampling (Pooling) Layer
model.add(AveragePooling2D(pool_size=(2, 2), strides=2))

# C5: Convolutional Layer
model.add(Conv2D(120, kernel_size=(5, 5), activation='relu'))

# Flatten the output for fully connected layers
model.add(Flatten())

# F6: Fully Connected Layer
model.add(Dense(84, activation='relu'))

# Output Layer (for binary classification, change units for multiclass)
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model Summary
model.summary()


d:\IDSGAN\IDSGAN-main\IDSGAN\myenv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 28, 6)      │           156 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ (None, 14, 14, 6)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 10, 10, 16)     │         2,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_1             │ (None, 5, 5, 16)       │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 1, 1, 120)      │        48,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 120)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 84)             │        10,164 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │         1,275 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 62,131 (242.70 KB)

 Trainable params: 62,131 (242.70 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
history = model.fit(
    X_train_CNN_7_layers, 
    y_train, 
    epochs=10, 
    batch_size=64, 
    validation_data=(X_test_CNN_7_layers, y_test),
    verbose=1
)

# model.save('CNN_7_layers.h5')
model.save('CNN_7_layers_pretrain.h5')

Epoch 1/10
28973/28973 ━━━━━━━━━━━━━━━━━━━━ 119s 4ms/step - accuracy: 0.8413 - loss: 0.5261 - val_accuracy: 0.8691 - val_loss: 0.3883
Epoch 2/10
28973/28973 ━━━━━━━━━━━━━━━━━━━━ 107s 4ms/step - accuracy: 0.8725 - loss: 0.3808 - val_accuracy: 0.8745 - val_loss: 0.3684
Epoch 3/10
28973/28973 ━━━━━━━━━━━━━━━━━━━━ 107s 4ms/step - accuracy: 0.8762 - loss: 0.3647 - val_accuracy: 0.8770 - val_loss: 0.3608
Epoch 4/10
28973/28973 ━━━━━━━━━━━━━━━━━━━━ 108s 4ms/step - accuracy: 0.8782 - loss: 0.3577 - val_accuracy: 0.8803 - val_loss: 0.3520
Epoch 5/10
28973/28973 ━━━━━━━━━━━━━━━━━━━━ 105s 4ms/step - accuracy: 0.8809 - loss: 0.3498 - val_accuracy: 0.8860 - val_loss: 0.3409
Epoch 6/10
28973/28973 ━━━━━━━━━━━━━━━━━━━━ 106s 4ms/step - accuracy: 0.8873 - loss: 0.3344 - val_accuracy: 0.8877 - val_loss: 0.3322
Epoch 7/10
28973/28973 ━━━━━━━━━━━━━━━━━━━━ 105s 4ms/step - accuracy: 0.8888 - loss: 0.3299 - val_accuracy: 0.8885 - val_loss: 0.3304
Epoch 8/10
28973/28973 ━━━━━━━━━━━━━━━━━━━━ 107s 4ms/step - ac

In [13]:
y_pred = model.predict(X_test_CNN_7_layers)

y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(y_test, axis=1)
print(y_pred.shape)
print(y_test.shape)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
def print_score(y_pred, y_real):
    print(f"Accuracy: {accuracy_score(y_real, y_pred):.4f}")
    print(f"Precision: {precision_score(y_real, y_pred, average='weighted'):.4f}")
    print(f"Recall: {recall_score(y_real, y_pred, average='weighted'):.4f}")
    print(f"F1: {f1_score(y_real, y_pred, average='weighted'):.4f}")
print_score(y_pred, y_test)


11936/11936 ━━━━━━━━━━━━━━━━━━━━ 17s 1ms/step
(381935,)
(381935,)
Accuracy: 0.9480
Precision: 0.9617
Recall: 0.9480
F1: 0.9425


**LSTM**

In [11]:
# Convert X_train and X_test dimension to 3-dimensional (row, 1, column)
X_train_LSTM = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test_LSTM = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

In [10]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score

model = Sequential()
model.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=(1, 96)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(32)))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='sigmoid'))  # Binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


d:\IDSGAN\IDSGAN-main\IDSGAN\myenv\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [12]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
history = model.fit(X_train_LSTM, y_train, epochs=15, batch_size=64, validation_split=0.1, callbacks=[early_stop])
# model.save('LSTM.h5')
model.save('LSTM_pretrain.h5')

Epoch 1/15
26076/26076 ━━━━━━━━━━━━━━━━━━━━ 65s 2ms/step - accuracy: 0.8503 - loss: 0.0547 - val_accuracy: 0.8781 - val_loss: 0.0376
Epoch 2/15
26076/26076 ━━━━━━━━━━━━━━━━━━━━ 63s 2ms/step - accuracy: 0.8766 - loss: 0.0385 - val_accuracy: 0.8809 - val_loss: 0.0366
Epoch 3/15
26076/26076 ━━━━━━━━━━━━━━━━━━━━ 68s 3ms/step - accuracy: 0.8796 - loss: 0.0374 - val_accuracy: 0.8824 - val_loss: 0.0358
Epoch 4/15
26076/26076 ━━━━━━━━━━━━━━━━━━━━ 60s 2ms/step - accuracy: 0.8819 - loss: 0.0366 - val_accuracy: 0.8846 - val_loss: 0.0350
Epoch 5/15
26076/26076 ━━━━━━━━━━━━━━━━━━━━ 60s 2ms/step - accuracy: 0.8841 - loss: 0.0360 - val_accuracy: 0.8872 - val_loss: 0.0346
Epoch 6/15
26076/26076 ━━━━━━━━━━━━━━━━━━━━ 60s 2ms/step - accuracy: 0.8858 - loss: 0.0354 - val_accuracy: 0.8874 - val_loss: 0.0343
Epoch 7/15
26076/26076 ━━━━━━━━━━━━━━━━━━━━ 61s 2ms/step - accuracy: 0.8862 - loss: 0.0352 - val_accuracy: 0.8876 - val_loss: 0.0343
Epoch 8/15
26076/26076 ━━━━━━━━━━━━━━━━━━━━ 61s 2ms/step - accuracy: 

**CNN + GRU**

In [12]:
X_train_CNN_GRU = np.zeros((X_train.shape[0], 32 * 32))
X_train_CNN_GRU[:, :96] = X_train  # Fill in the first 96 elements

# Reshape to (390500, 32, 32, 1) for the CNN
X_train_CNN_GRU = X_train_CNN_GRU.reshape(-1, 32, 32, 1)

X_test_CNN_GRU = np.zeros((X_test.shape[0], 32 * 32))
X_test_CNN_GRU[:, :96] = X_test  # Fill in the first 97 elements

# Reshape to (390500, 32, 32, 1) for the CNN
X_test_CNN_GRU = X_test_CNN_GRU.reshape(-1, 32, 32, 1)

In [13]:
#CNN_GRU
#Network Intrusion Detection Model Based on CNN and GRU
from keras.layers import GRU
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, LSTM, GRU, MaxPooling2D, Conv2D

# model = Sequential()
# model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape))
# model.add(MaxPooling1D(pool_size=2))
# # model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
# # model.add(MaxPooling1D(pool_size=2))
# model.add(GRU(32, return_sequences=False))
# model.add(Flatten())
# # model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(num_classes, activation='softmax'))
# model.summary()

model = Sequential()

    # CNN Layers for spatial feature extraction
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
    #  output to feed into GRU layers
model.add(Flatten())
model.add(Reshape((model.output_shape[1], 1)))  # Reshape for GRU 
    # GRU or temporal sequence learning
model.add(GRU(64, activation='relu', return_sequences=True))
model.add(GRU(32, activation='relu'))
    # Conected layer for classification
    
model.add(Dropout(0.5))  # Dropout for regularization
model.add(Dense(num_classes, activation='softmax'))  # Softmax for multi-class

model.summary()


d:\IDSGAN\IDSGAN-main\IDSGAN\myenv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 30, 30, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 4, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 2, 2, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 512, 1)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 512, 64)        │        12,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 32)             │         9,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 15)             │           495 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 115,439 (450.93 KB)

 Trainable params: 115,439 (450.93 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import tensorflow as tf
opt = Adam(learning_rate=0.001)
model.compile(optimizer='adam', loss= tf.keras.metrics.categorical_crossentropy,
                  metrics=['accuracy'])


from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, mode="min", verbose=1, min_lr=0)
#plotlosses = PlotLossesKeras()
call_backs = [lr_reduce]
EPOCHS = 10
BATCH_SIZE = 256

history = model.fit(X_train_CNN_GRU, y_train,
                   validation_data=(X_test_CNN_GRU, y_test),
                    validation_split=0.1,
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE,
                    callbacks=call_backs,
                    #class_weight=class_weights,
                    verbose=1)
model.save('CNN_GRU.h5')

In [ ]:
os.chdir("/content/drive/MyDrive/IDSGAN/Edge-IIoT/")
model.save('CNN_GRU.h5')

**Evaluation**

In [6]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
def print_score(y_pred, y_real):
    print(f"Accuracy: {accuracy_score(y_real, y_pred):.4f}")
    print(f"Precision: {precision_score(y_real, y_pred, average='weighted'):.4f}")
    print(f"Recall: {recall_score(y_real, y_pred, average='weighted'):.4f}")
    print(f"F1: {f1_score(y_real, y_pred, average='weighted'):.4f}")


In [22]:
#CNN with 3 layers
# CNN_3_layers = tf.keras.models.load_model('d:\\IDSGAN\\IDSGAN-main\\IDSGAN\\Dataset\\Edge-IIoTset\\CNN_3_layers.h5')
CNN_3_layers = tf.keras.models.load_model('d:\\IDSGAN\\IDSGAN-main\\IDSGAN\\Dataset\\Edge-IIoTset\\CNN_3_layers_pretrain.h5')
y_pred = CNN_3_layers.predict(X_test_CNN_3_layers)
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print_score(y_pred, y_true)

11936/11936 ━━━━━━━━━━━━━━━━━━━━ 17s 1ms/step
Accuracy: 0.9420
Precision: 0.9541
Recall: 0.9420
F1: 0.9349


In [21]:
#CNN with 5 layers
CNN_5_layers = tf.keras.models.load_model('d:\\IDSGAN\\IDSGAN-main\\IDSGAN\\Dataset\\Edge-IIoTset\\CNN_5_layers.h5')
# CNN_5_layers = tf.keras.models.load_model('d:\\IDSGAN\\IDSGAN-main\\IDSGAN\\Dataset\\Edge-IIoTset\\CNN_5_layers_pretrain.h5')
y_pred = CNN_5_layers.predict(X_test_CNN_5_layers)
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print_score(y_pred, y_true)

11936/11936 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step
Accuracy: 0.9493
Precision: 0.9547
Recall: 0.9493
F1: 0.9460


In [23]:
#CNN with 7 layers
# CNN_7_layers = tf.keras.models.load_model('d:\\IDSGAN\\IDSGAN-main\\IDSGAN\\Dataset\\Edge-IIoTset\\CNN_7_layers.h5')
CNN_7_layers = tf.keras.models.load_model('d:\\IDSGAN\\IDSGAN-main\\IDSGAN\\Dataset\\Edge-IIoTset\\CNN_7_layers_pretrain.h5')
y_pred = CNN_7_layers.predict(X_test_CNN_7_layers)
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print_score(y_pred, y_true)

11936/11936 ━━━━━━━━━━━━━━━━━━━━ 24s 2ms/step
Accuracy: 0.9438
Precision: 0.9475
Recall: 0.9438
F1: 0.9422


In [24]:
#LSTM
# LSTM = tf.keras.models.load_model('d:\\IDSGAN\\IDSGAN-main\\IDSGAN\\Dataset\\Edge-IIoTset\\LSTM.h5')
LSTM = tf.keras.models.load_model('d:\\IDSGAN\\IDSGAN-main\\IDSGAN\\Dataset\\Edge-IIoTset\\LSTM_pretrain.h5')
y_pred = LSTM.predict(X_test_LSTM)
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print_score(y_pred, y_true)

11936/11936 ━━━━━━━━━━━━━━━━━━━━ 18s 1ms/step
Accuracy: 0.9481
Precision: 0.9506
Recall: 0.9481
F1: 0.9470


In [ ]:
#CNN+GRU
CNN_GRU = tf.keras.models.load_model('d:\\IDSGAN\\IDSGAN-main\\IDSGAN\\Dataset\\Edge-IIoTset\\CNN_GRU.h5')
y_pred = CNN_GRU.predict(X_test_CNN_GRU)
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print_score(y_pred, y_true)

11936/11936 ━━━━━━━━━━━━━━━━━━━━ 33s 3ms/step
Accuracy: 0.9324
Precision: 0.9216


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Recall: 0.9324
F1: 0.9195


**CTGAN**

In [ ]:
from sdv.metadata import SingleTableMetadata

# Đọc dữ liệu từ file CSV
real_data = pd.read_csv('C:\\Users\\DUCLUONG\\Downloads\\DNN-EdgeIIoT-dataset.csv\\DNN-EdgeIIoT-dataset.csv', low_memory=False)

# Khởi tạo đối tượng metadata
metadata = SingleTableMetadata()

# Phát hiện metadata từ DataFrame
metadata.detect_from_dataframe(real_data)

# Kiểm tra dữ liệu và metadata
#print(real_data.head())
#print(metadata.to_dict())

In [ ]:
metadata.get_column_names(sdtype='categorical')

In [ ]:
from sklearn.utils import shuffle

# Danh sách các cột cần loại bỏ
drop_columns = ["frame.time", "ip.src_host", "ip.dst_host", "arp.src.proto_ipv4","arp.dst.proto_ipv4",
                "http.file_data","http.request.full_uri","icmp.transmit_timestamp",
                "http.request.uri.query", "tcp.options","tcp.payload","tcp.srcport",
                "tcp.dstport", "udp.port", "mqtt.msg"]

# Loại bỏ các cột (bỏ qua những cột không có trong DataFrame)
real_data.drop(drop_columns, axis=1, inplace=True, errors='ignore')

# Loại bỏ các hàng có giá trị null và trùng lặp
real_data.dropna(axis=0, how='any', inplace=True)
real_data.drop_duplicates(subset=None, keep="first", inplace=True)

# Shuffle dữ liệu
real_data = shuffle(real_data)

In [ ]:
from sdv.metadata import SingleTableMetadata

# Tạo lại metadata dựa trên dữ liệu đã loại bỏ cột
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(real_data)

In [ ]:
from sdv.single_table import CTGANSynthesizer

# Khởi tạo mô hình với batch_size tùy chỉnh
synthesizer = CTGANSynthesizer(metadata, batch_size=100, epochs= 50)

# Huấn luyện mô hình
synthesizer.fit(real_data)

# Lưu mô hình đã huấn luyện
synthesizer.save('C:\\Users\\DUCLUONG\\Downloads\\ctgan_model_2.pkl')

In [ ]:
from sdv.single_table import CTGANSynthesizer

synthesizer = CTGANSynthesizer.load(
    filepath='C:\\Users\\DUCLUONG\\Downloads\\ctgan_model_2.pkl'
)

In [ ]:
from sdv.sampling import Condition

conditions = [
    Condition(
        num_rows=5380,
        column_values={'Attack_type': 'Normal'}
    ),
    Condition(
        num_rows=5380,
        column_values={'Attack_type': 'MITM'}
    ),
    Condition(
        num_rows=5380,
        column_values={'Attack_type': 'Uploading'}
    ),
    Condition(
        num_rows=5380,
        column_values={'Attack_type': 'Ransomware'}
    ),
    Condition(
        num_rows=5380,
        column_values={'Attack_type': 'SQL_injection'}
    ),
    Condition(
        num_rows=5380,
        column_values={'Attack_type': 'DDoS_HTTP'}
    ),
    Condition(
        num_rows=5380,
        column_values={'Attack_type': 'DDoS_TCP'}
    ),
    Condition(
        num_rows=5380,
        column_values={'Attack_type': 'Password'}
    ),
    Condition(
        num_rows=5380,
        column_values={'Attack_type': 'Port_Scanning'}
    ),
    Condition(
        num_rows=5380,
        column_values={'Attack_type': 'Vulnerability_scanner'}
    ),
    Condition(
        num_rows=5380,
        column_values={'Attack_type': 'Backdoor'}
    ),
    Condition(
        num_rows=5380,
        column_values={'Attack_type': 'XSS'}
    ),
    Condition(
        num_rows=5380,
        column_values={'Attack_type': 'Fingerprinting'}
    ),
    Condition(
        num_rows=5380,
        column_values={'Attack_type': 'DDoS_UDP'}
    ),
    Condition(
        num_rows=5380,
        column_values={'Attack_type': 'DDoS_ICMP'}
    ),
    
    # http.request.method điều kiện
    Condition(
        num_rows=5380,
        column_values={'http.request.method': '0.0'}
    ),
    Condition(
        num_rows=5380,
        column_values={'http.request.method': '0'}
    ),
    Condition(
        num_rows=5380,
        column_values={'http.request.method': 'GET'}
    ),
    Condition(
        num_rows=5380,
        column_values={'http.request.method': 'TRACE'}
    ),
    Condition(
        num_rows=5380,
        column_values={'http.request.method': 'POST'}
    ),
    Condition(
        num_rows=5380,
        column_values={'http.request.method': 'OPTIONS'}
    ),
    Condition(
        num_rows=5380,
        column_values={'http.request.method': 'PUT'}
    ),
    Condition(
        num_rows=5380,
        column_values={'http.request.method': 'PROPFIND'}
    ),
    Condition(
        num_rows=5380,
        column_values={'http.request.method': 'SEARCH'}
    ),
    
    # http.referer điều kiện
    Condition(
        num_rows=5380,
        column_values={'http.referer': '0.0'}
    ),
    Condition(
        num_rows=5380,
        column_values={'http.referer': '0'}
    ),
    Condition(
        num_rows=5380,
        column_values={'http.referer': '127.0.0.1'}
    ),
    Condition(
        num_rows=5380,
        column_values={'http.referer': '() { _; } >_[$($())] { echo 93e4r0-CVE-2014-6278: true; echo;echo; }'}
    ),
    Condition(
        num_rows=5380,
        column_values={'http.referer': 'TESTING_PURPOSES_ONLY'}
    ),
    
    # http.request.version điều kiện
    Condition(
        num_rows=5380,
        column_values={'http.request.version': '0.0'}
    ),
    Condition(
        num_rows=5380,
        column_values={'http.request.version': '0'}
    ),
    Condition(
        num_rows=5380,
        column_values={'http.request.version': 'HTTP/1.1'}
    ),
    Condition(
        num_rows=5380,
        column_values={'http.request.version': 'HTTP/1.0'}
    ),
    Condition(
        num_rows=5380,
        column_values={'http.request.version': 'Src=javascript:alert(\'Vulnerable\')><Img Src=\\" HTTP/1.1'}
    ),
    Condition(
        num_rows=5380,
        column_values={'http.request.version': 'script>alert(1)/script><\\" HTTP/1.1'}
    ),
    Condition(
        num_rows=5380,
        column_values={'http.request.version': '-al&ABSOLUTE_PATH_STUDIP=http://cirt.net/rfiinc.txt?? HTTP/1.1'}
    ),
    Condition(
        num_rows=5380,
        column_values={'http.request.version': '-a HTTP/1.1'}
    ),
    Condition(
        num_rows=5380,
        column_values={'http.request.version': 'By Dr HTTP/1.1'}
    ),
    Condition(
        num_rows=5380,
        column_values={'http.request.version': '/etc/passwd|?data=Download HTTP/1.1'}
    ),
    Condition(
        num_rows=5380,
        column_values={'http.request.version': '-al&_PHPLIB[libdir]=http://cirt.net/rfiinc.txt?? HTTP/1.1'}
    ),
    Condition(
        num_rows=5380,
        column_values={'http.request.version': '> HTTP/1.1'}
    ),
    Condition(
        num_rows=5380,
        column_values={'http.request.version': "name=a><input name=i value=XSS>&lt;script>alert('Vulnerable')</script> HTTP/1.1"}
    ),
    
    # dns.qry.name.len điều kiện
    Condition(
        num_rows=5380,
        column_values={'dns.qry.name.len': '0'}
    ),
    Condition(
        num_rows=5380,
        column_values={'dns.qry.name.len': '0.0'}
    ),
    Condition(
        num_rows=5380,
        column_values={'dns.qry.name.len': '2.debian.pool.ntp.org'}
    ),
    Condition(
        num_rows=5380,
        column_values={'dns.qry.name.len': '3.debian.pool.ntp.org'}
    ),
    Condition(
        num_rows=5380,
        column_values={'dns.qry.name.len': '0.debian.pool.ntp.org'}
    ),
    Condition(
        num_rows=5380,
        column_values={'dns.qry.name.len': '1.debian.pool.ntp.org'}
    ),
    Condition(
        num_rows=5380,
        column_values={'dns.qry.name.len': '1.0'}
    ),
    Condition(
        num_rows=5380,
        column_values={'dns.qry.name.len': 'raspberrypi.local'}
    ),
    Condition(
        num_rows=5380,
        column_values={'dns.qry.name.len': 'null-null.local'}
    ),
    Condition(
        num_rows=5380,
        column_values={'dns.qry.name.len': '_googlecast._tcp.local'}
    ),
    
    # mqtt.conack.flags điều kiện
    Condition(
        num_rows=5380,
        column_values={'mqtt.conack.flags': '0x00000000'}
    ),
    Condition(
        num_rows=5380,
        column_values={'mqtt.conack.flags': '0'}
    ),
    Condition(
        num_rows=5380,
        column_values={'mqtt.conack.flags': '0.0'}
    ),
    Condition(
        num_rows=5380,
        column_values={'mqtt.conack.flags': '1574358'}
    ),
    Condition(
        num_rows=5380,
        column_values={'mqtt.conack.flags': '1471198'}
    ),
    Condition(
        num_rows=5380,
        column_values={'mqtt.conack.flags': '1574359'}
    ),
    Condition(
        num_rows=5380,
        column_values={'mqtt.conack.flags': '1461384'}
    ),
    Condition(
        num_rows=5380,
        column_values={'mqtt.conack.flags': '1471199'}
    ),
    Condition(
        num_rows=5380,
        column_values={'mqtt.conack.flags': '1461589'}
    ),
    Condition(
        num_rows=5380,
        column_values={'mqtt.conack.flags': '1461383'}
    ),
    Condition(
        num_rows=5380,
        column_values={'mqtt.conack.flags': '1461073'}
    ),
    Condition(
        num_rows=5380,
        column_values={'mqtt.conack.flags': '1461591'}
    ),
    Condition(
        num_rows=5380,
        column_values={'mqtt.conack.flags': '1461074'}
    ),
    
    # mqtt.protoname điều kiện
    Condition(
        num_rows=5380,
        column_values={'mqtt.protoname': '0'}
    ),
    Condition(
        num_rows=5380,
        column_values={'mqtt.protoname': '0.0'}
    ),
    Condition(
        num_rows=5380,
        column_values={'mqtt.protoname': 'MQTT'}
    ),
    
    # mqtt.topic điều kiện
    Condition(
        num_rows=5380,
        column_values={'mqtt.topic': '0'}
    ),
    Condition(
        num_rows=5380,
        column_values={'mqtt.topic': '0.0'}
    ),
    Condition(
        num_rows=5380,
        column_values={'mqtt.topic': 'Temperature_and_Humidity'}
    )
]


synthetic_data = synthesizer.sample_from_conditions(
    conditions=conditions,
    batch_size=500,  
    max_tries_per_batch=500,  
    output_file_path='C:\\Users\\DUCLUONG\\Downloads\\synthetic-data_50ep.csv'
)


# synthetic_data.to_csv('C:\\Users\\DUCLUONG\\Downloads\\synthetic_data_CS.csv', index=False)

 **CTGAN evaluation**

In [20]:
import pandas as pd
#real_data = pd.read_csv("d:\\IDSGAN\\IDSGAN-main\\IDSGAN\\Dataset\\Edge-IIoTset\\preprocessed.csv", low_memory=False)
# GAN_df = pd.read_csv("d:\\IDSGAN\\IDSGAN-main\\IDSGAN\\Dataset\\Edge-IIoTset\\synthetic_data_CS.csv", low_memory=False)
GAN_df = pd.read_csv("d:\\IDSGAN\\IDSGAN-main\\IDSGAN\\Dataset\\Edge-IIoTset\\synthetic_data_15ep.csv", low_memory=False)


In [3]:
def encode_text_dummy(GAN_df, name):
    dummies = pd.get_dummies(GAN_df[name])
    for x in dummies.columns:
        dummy_name = f"{name}-{x}"
        GAN_df[dummy_name] = dummies[x]
    GAN_df.drop(name, axis=1, inplace=True)

encode_text_dummy(GAN_df,'http.request.method')
encode_text_dummy(GAN_df,'http.referer')
encode_text_dummy(GAN_df,"http.request.version")
encode_text_dummy(GAN_df,"dns.qry.name.len")
encode_text_dummy(GAN_df,"mqtt.conack.flags")
encode_text_dummy(GAN_df,"mqtt.protoname")
encode_text_dummy(GAN_df,"mqtt.topic")

In [ ]:
# Apply Median Filtering to all features
window_size_median = 3  # Window size for median filtering
for feature in GAN_df.columns:
    if feature != 'Attack_type':  # Exclude the label column
        GAN_df[feature + '_median_filtered'] = GAN_df[feature].rolling(window=window_size_median, center=True).median()

# Apply Standard Deviation-based Filtering to all features
threshold_std = 3  # Threshold for standard deviation-based filtering
for feature in GAN_df.columns:
    if feature != 'Attack_type':  # Exclude the label column
        mean = GAN_df[feature].mean()
        std = GAN_df[feature].std()
        upper_bound = mean + threshold_std * std
        lower_bound = mean - threshold_std * std
        GAN_df[feature + '_std_filtered'] = GAN_df[feature].apply(lambda x: x if (lower_bound <= x <= upper_bound) else None)

# Drop original features that were filtered
GAN_df.drop(GAN_df.columns[GAN_df.columns.str.endswith('_filtered')], axis=1, inplace=True)

In [24]:
GAN_df.to_csv('CTGAN_preprocessed_15ep.csv', index=False)

In [5]:
# GAN_df = pd.read_csv("CTGAN_preprocessed.csv", low_memory=False)
GAN_df = pd.read_csv("CTGAN_preprocessed_15ep.csv", low_memory=False)
y_test = GAN_df["Attack_type"]
X_test = GAN_df.drop(["Attack_type"], axis=1)

In [6]:
# Converting categorical value in "Attack_type" to integer value
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_test = label_encoder.fit_transform(y_test)

label_encoder.classes_

#Normalize
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()
X_test = min_max_scaler.fit_transform(X_test)

**ASR with CNN 3 layers**

In [8]:
X_test_CNN_3_layers = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
X_test_CNN_3_layers.shape

(381935, 96, 1)

In [9]:
#CNN_3_layers
import tensorflow as tf
model = tf.keras.models.load_model('d:\\IDSGAN\\IDSGAN-main\\IDSGAN\\Dataset\\Edge-IIoTset\\CNN_3_layers.h5')
# model = tf.keras.models.load_model('d:\\IDSGAN\\IDSGAN-main\\IDSGAN\\Dataset\\Edge-IIoTset\\CNN_3_layers_pretrain.h5')
y_pred = model.predict(X_test_CNN_3_layers)
y_pred = np.argmax(y_pred, axis=1)

print(np.shape(y_test))
print(np.shape(y_pred))

#Calculate ASR

original_labels = np.array(y_test)
attacked_labels = np.array(y_pred)

# Calculate misclassified label
num_misclassified = np.sum(original_labels != attacked_labels)

# Total attack samples
total_samples = len(original_labels)

asr = num_misclassified / total_samples
print(f"ASR on CNN 3 layers IDS: {asr:.2%}")


12204/12204 ━━━━━━━━━━━━━━━━━━━━ 12s 989us/step
(390500,)
(390500,)
ASR on CNN 3 layers IDS: 59.10%


In [10]:
#CNN_3_layers
import tensorflow as tf
# model = tf.keras.models.load_model('d:\\IDSGAN\\IDSGAN-main\\IDSGAN\\Dataset\\Edge-IIoTset\\CNN_3_layers_pretrain.h5')
model = tf.keras.models.load_model('d:\\IDSGAN\\IDSGAN-main\\IDSGAN\\Dataset\\Edge-IIoTset\\CNN_3_layers_pretrain.h5')
y_pred = model.predict(X_test_CNN_3_layers)
y_pred = np.argmax(y_pred, axis=1)

print(np.shape(y_test))
print(np.shape(y_pred))

#Calculate ASR

original_labels = np.array(y_test)
attacked_labels = np.array(y_pred)

# Calculate misclassified label
num_misclassified = np.sum(original_labels != attacked_labels)

# Total attack samples
total_samples = len(original_labels)

asr = num_misclassified / total_samples
print(f"ASR on CNN 3 layers IDS: {asr:.2%}")


12204/12204 ━━━━━━━━━━━━━━━━━━━━ 12s 960us/step
(390500,)
(390500,)
ASR on CNN 3 layers IDS: 59.49%


**ASR with CNN 5 layers**

In [9]:
X_test_CNN_5_layers = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
X_test_CNN_5_layers.shape

(381935, 96, 1)

In [12]:
#CNN_5_layers
import tensorflow as tf
model = tf.keras.models.load_model('d:\\IDSGAN\\IDSGAN-main\\IDSGAN\\Dataset\\Edge-IIoTset\\CNN_5_layers.h5')
# model = tf.keras.models.load_model('d:\\IDSGAN\\IDSGAN-main\\IDSGAN\\Dataset\\Edge-IIoTset\\CNN_5_layers_pretrain.h5')
y_pred = model.predict(X_test_CNN_5_layers)
y_pred = np.argmax(y_pred, axis=1)

print(np.shape(y_test))
print(np.shape(y_pred))

#Calculate ASR

original_labels = np.array(y_test)
attacked_labels = np.array(y_pred)

# Calculate misclassified label
num_misclassified = np.sum(original_labels != attacked_labels)

# Total attack samples
total_samples = len(original_labels)

asr = num_misclassified / total_samples
print(f"ASR on CNN 5 layers IDS: {asr:.2%}")


12204/12204 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step
(390500,)
(390500,)
ASR on CNN 5 layers IDS: 35.46%


In [13]:
#CNN_5_layers
import tensorflow as tf
# model = tf.keras.models.load_model('d:\\IDSGAN\\IDSGAN-main\\IDSGAN\\Dataset\\Edge-IIoTset\\CNN_5_layers.h5')
model = tf.keras.models.load_model('d:\\IDSGAN\\IDSGAN-main\\IDSGAN\\Dataset\\Edge-IIoTset\\CNN_5_layers_pretrain.h5')
y_pred = model.predict(X_test_CNN_5_layers)
y_pred = np.argmax(y_pred, axis=1)

print(np.shape(y_test))
print(np.shape(y_pred))

#Calculate ASR

original_labels = np.array(y_test)
attacked_labels = np.array(y_pred)

# Calculate misclassified label
num_misclassified = np.sum(original_labels != attacked_labels)

# Total attack samples
total_samples = len(original_labels)

asr = num_misclassified / total_samples
print(f"ASR on CNN 5 layers IDS: {asr:.2%}")


12204/12204 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step
(390500,)
(390500,)
ASR on CNN 5 layers IDS: 35.79%


**ASR with CNN 7 layers**

In [10]:
X_test_CNN_7_layers = np.zeros((X_test.shape[0], 32 * 32))
X_test_CNN_7_layers[:, :96] = X_test  # Fill in the first 97 elements

# Reshape to (390500, 32, 32, 1) for the CNN
X_test_CNN_7_layers = X_test_CNN_7_layers.reshape(-1, 32, 32, 1)

In [15]:
#CNN_7_layers
import tensorflow as tf
model = tf.keras.models.load_model('d:\\IDSGAN\\IDSGAN-main\\IDSGAN\\Dataset\\Edge-IIoTset\\CNN_7_layers.h5')
y_pred = model.predict(X_test_CNN_7_layers)
y_pred = np.argmax(y_pred, axis=1)

print(np.shape(y_test))
print(np.shape(y_pred))

#Calculate ASR

original_labels = np.array(y_test)
attacked_labels = np.array(y_pred)

# Calculate misclassified label
num_misclassified = np.sum(original_labels != attacked_labels)

# Total attack samples
total_samples = len(original_labels)

asr = num_misclassified / total_samples
print(f"ASR on CNN 7 layers IDS: {asr:.2%}")


12204/12204 ━━━━━━━━━━━━━━━━━━━━ 18s 1ms/step
(390500,)
(390500,)
ASR on CNN 7 layers IDS: 40.37%


In [16]:
#CNN_7_layers pretrained
import tensorflow as tf
model = tf.keras.models.load_model('d:\\IDSGAN\\IDSGAN-main\\IDSGAN\\Dataset\\Edge-IIoTset\\CNN_7_layers_pretrain.h5')
y_pred = model.predict(X_test_CNN_7_layers)
y_pred = np.argmax(y_pred, axis=1)

print(np.shape(y_test))
print(np.shape(y_pred))

#Calculate ASR

original_labels = np.array(y_test)
attacked_labels = np.array(y_pred)

# Calculate misclassified label
num_misclassified = np.sum(original_labels != attacked_labels)

# Total attack samples
total_samples = len(original_labels)

asr = num_misclassified / total_samples
print(f"ASR on CNN 7 layers IDS: {asr:.2%}")


12204/12204 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step
(390500,)
(390500,)
ASR on CNN 7 layers IDS: 35.17%


**ASR with CNN + GRU**

In [31]:
X_test_CNN_GRU = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
X_test_CNN_GRU.shape

(390500, 96, 1)

In [ ]:
#CNN + GRU
import tensorflow as tf
model = tf.keras.models.load_model('d:\\IDSGAN\\IDSGAN-main\\IDSGAN\\Dataset\\Edge-IIoTset\\CNN_GRU.h5')
y_pred = model.predict(X_test_CNN_GRU)
y_pred = np.argmax(y_pred, axis=1)

print(np.shape(y_test))
print(np.shape(y_pred))

#Calculate ASR

original_labels = np.array(y_test)
attacked_labels = np.array(y_pred)

# Calculate misclassified label
num_misclassified = np.sum(original_labels != attacked_labels)

# Total attack samples
total_samples = len(original_labels)

asr = num_misclassified / total_samples
print(f"ASR on CNN+GRU based IDS: {asr:.2%}")


**LSTM**

In [17]:
X_test_LSTM = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
X_test_LSTM.shape

(381935, 1, 96)

In [18]:
#LSTM
import tensorflow as tf
model = tf.keras.models.load_model('d:\\IDSGAN\\IDSGAN-main\\IDSGAN\\Dataset\\Edge-IIoTset\\LSTM.h5')
# model = tf.keras.models.load_model('d:\\IDSGAN\\IDSGAN-main\\IDSGAN\\Dataset\\Edge-IIoTset\\LSTM_pretrain.h5')
y_pred = model.predict(X_test_LSTM)
y_pred = np.argmax(y_pred, axis=1)

print(np.shape(y_test))
print(np.shape(y_pred))

#Calculate ASR

original_labels = np.array(y_test)
attacked_labels = np.array(y_pred)

# Calculate misclassified label
num_misclassified = np.sum(original_labels != attacked_labels)

# Total attack samples
total_samples = len(original_labels)

asr = num_misclassified / total_samples
print(f"ASR on LSTM-based IDS: {asr:.2%}")


12204/12204 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step
(390500,)
(390500,)
ASR on LSTM-based IDS: 33.94%


In [19]:
#LSTM
import tensorflow as tf
# model = tf.keras.models.load_model('d:\\IDSGAN\\IDSGAN-main\\IDSGAN\\Dataset\\Edge-IIoTset\\LSTM.h5')
model = tf.keras.models.load_model('d:\\IDSGAN\\IDSGAN-main\\IDSGAN\\Dataset\\Edge-IIoTset\\LSTM_pretrain.h5')
y_pred = model.predict(X_test_LSTM)
y_pred = np.argmax(y_pred, axis=1)

print(np.shape(y_test))
print(np.shape(y_pred))

#Calculate ASR

original_labels = np.array(y_test)
attacked_labels = np.array(y_pred)

# Calculate misclassified label
num_misclassified = np.sum(original_labels != attacked_labels)

# Total attack samples
total_samples = len(original_labels)

asr = num_misclassified / total_samples
print(f"ASR on LSTM-based IDS: {asr:.2%}")


12204/12204 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step
(390500,)
(390500,)
ASR on LSTM-based IDS: 36.20%


**Defense Method**

MANDA

In [ ]:
#MANDA
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Ensure reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Load the dataset
df = pd.read_csv('preprocessed.csv')  # Real dataset
GAN_df = pd.read_csv('preprocessed_gan.csv')  # Synthetic dataset

# Prepare features and labels
X_real = df.drop(columns=['Attack_type']).values
y_real = df['Attack_type'].values
X_gan = GAN_df.drop(columns=['Attack_type']).values
y_gan = GAN_df['Attack_type'].values

# Standardize features
scaler = StandardScaler()
X_real = scaler.fit_transform(X_real)
X_gan = scaler.transform(X_gan)

# Encode labels
def encode_labels(y):
    le = LabelEncoder()
    return le.fit_transform(y), le

y_real, label_encoder = encode_labels(y_real)
y_gan = label_encoder.transform(y_gan)

# Load pre-trained IDS model (CNN + GRU)
ids_model = load_model('C:\\Users\\DUCLUONG\\Downloads\\CNN_GRU.h5')
#ids_model = load_model('C:\\Users\\DUCLUONG\\Downloads\\CNN_3_layers.h5')
#ids_model = load_model('C:\\Users\\DUCLUONG\\Downloads\\CNN_5_layers.h5')
#ids_model = load_model('C:\\Users\\DUCLUONG\\Downloads\\CNN_7_layers.h5')
#ids_model = load_model('C:\\Users\\DUCLUONG\\Downloads\\LSTM.h5')
# Reshape function for CNN-GRU input
def reshape_for_cnn_7(X, target_shape=(30, 30, 1)):
    """Reshape input to match Conv2D input shape with padding if needed."""
    num_features = target_shape[0] * target_shape[1]  # 30x30 = 900
    X_padded = pad_sequences(X, maxlen=num_features, padding='post', dtype='float32')
    X_padded = X_padded.reshape((X.shape[0], *target_shape))  # Reshape to (batch_size, 30, 30, 1)
    return X_padded

def batch_predict(model, X, batch_size=512):
    """Predict model outputs in batches to save memory."""
    preds = []
    for i in range(0, X.shape[0], batch_size):
        X_batch = X[i:i + batch_size]
        preds_batch = model.predict(X_batch, verbose=0)
        preds.append(preds_batch)
    return np.vstack(preds)

class MANDA:
    def __init__(self, manifold_model, threshold_std=3, threshold_boundary=0.1):
        self.manifold_model = manifold_model  # Strong manifold-based model
        self.threshold_std = threshold_std
        self.threshold_boundary = threshold_boundary

    def compute_manifold_score(self, X, model_probs):
        """Compute manifold-based inconsistency scores."""
        scores = []
        batch_size = 512
        for i in range(0, X.shape[0], batch_size):
            X_batch = X[i:i + batch_size]
            manifold_probs = self.manifold_model.predict_proba(X_batch)
            score_batch = np.abs(model_probs[i:i + batch_size] - manifold_probs).sum(axis=1)
            scores.extend(score_batch)
        return np.array(scores)

    def compute_boundary_score(self, X, model):
        """Compute decision boundary scores based on perturbation."""
        scores = []
        batch_size = 512
        for i in range(0, X.shape[0], batch_size):
            X_batch = X[i:i + batch_size]
            perturbations = np.random.normal(0, 0.01, X_batch.shape)
            X_perturbed = X_batch + perturbations

            X_batch_reshaped = reshape_for_cnn_7(X_batch)
            X_perturbed_reshaped = reshape_for_cnn_7(X_perturbed)

            orig_preds = model.predict(X_batch_reshaped, verbose=0)
            perturbed_preds = model.predict(X_perturbed_reshaped, verbose=0)
            score_batch = np.linalg.norm(orig_preds - perturbed_preds, axis=1)
            scores.extend(score_batch)
        return np.array(scores)

    def detect_adversarial(self, X, y, model):
        """Detect adversarial examples."""
        X_reshaped = reshape_for_cnn_7(X)
        model_probs = batch_predict(model, X_reshaped)
        predictions = np.argmax(model_probs, axis=1)

        # Compute scores
        score1 = self.compute_manifold_score(X, model_probs)
        score2 = self.compute_boundary_score(X, model)

        # Áp dụng ngưỡng để xác định adversarial examples
        threshold_score1 = np.percentile(score1, 95)  # Ngưỡng dựa trên phân vị 95%
        threshold_score2 = np.percentile(score2, 95)

        adversarial_flags = (score1 > threshold_score1) | (score2 > threshold_score2)

        # Combine scores for logistic regression
        combined_scores = np.vstack([score1, score2]).T
        is_adversarial = adversarial_flags.astype(int)
        return is_adversarial, combined_scores


    def train_combined_model(self, X_clean, y_clean, X_adv, y_adv, model):
        """Train logistic regression to combine manifold and decision boundary scores."""
        # Combine clean and adversarial data
        X_combined = np.vstack([X_clean, X_adv])
        y_combined = np.hstack([np.zeros(len(X_clean)), np.ones(len(X_adv))])

        # Compute probabilities and scores
        X_combined_reshaped = reshape_for_cnn_7(X_combined)
        model_probs = batch_predict(model, X_combined_reshaped)

        score1 = self.compute_manifold_score(X_combined, model_probs)
        score2 = self.compute_boundary_score(X_combined, model)
        combined_scores = np.vstack([score1, score2]).T

        # Train logistic regression
        self.logistic_model = LogisticRegression()
        self.logistic_model.fit(combined_scores, y_combined)

# Train a Random Forest model as the manifold model
manifold_model = RandomForestClassifier(n_estimators=100, random_state=42)
manifold_model.fit(X_real, y_real)

# Train MANDA
manda = MANDA(manifold_model=manifold_model)
manda.train_combined_model(X_real, y_real, X_gan, y_gan, ids_model)

# Detect adversarial examples
is_adversarial, combined_scores = manda.detect_adversarial(X_gan, y_gan, ids_model)

# Report results
print(f"Detected {is_adversarial.sum()} adversarial examples out of {len(X_gan)} samples.")


KDE

In [ ]:
import numpy as np
from sklearn.neighbors import KernelDensity
import joblib
# KDE-based detection
def kde_detect_adversarial(original_data, adversarial_data, bandwidth=1.0, threshold=0.1):
    """
    Detect adversarial attacks using Kernel Density Estimation (KDE).
    
    Args:
        original_data (numpy.ndarray): Clean data samples.
        adversarial_data (numpy.ndarray): Adversarial examples.
        bandwidth (float): Bandwidth for the KDE model.
        threshold (float): Threshold for detecting anomalies.
    
    Returns:
        bool: True if adversarial attack detected, False otherwise.
    """
    # Fit KDE model on original data
    # kde = KernelDensity(kernel='gaussian', bandwidth=bandwidth)
    # kde.fit(original_data)
    # joblib.dump(kde, 'kde_model.pkl')

    kde = joblib.load('kde_model.pkl')
    # Calculate log density for adversarial data
    log_density_adv = kde.score_samples(adversarial_data)
    
    # Compute mean log density of adversarial samples
    mean_log_density_adv = np.mean(log_density_adv)
    
    print(f"Mean Log Density (Adversarial): {mean_log_density_adv}")
    
    # Detect adversarial attacks based on threshold
    return mean_log_density_adv < threshold

# Example usage
# Simulated data
# original_data = np.random.rand(1000, 96)  # Replace with your original feature data
# adversarial_data = np.random.rand(1000, 96) + 0.1  # Replace with your adversarial examples
# adversarial_data = original_data[:20000]

# Detect adversarial attacks
if kde_detect_adversarial(original_data, adversarial_data, bandwidth=1.0, threshold=-200):
    print("Adversarial attack detected!")
else:
    print("No adversarial attack detected.")